In [39]:
import random
# Redo hand if three people enter the flop.

# Opponents = 6 max and 100 man tournament; 10 paid:   Just assume 9 max is 6 max table
    # 30, 20, 12, 9.5, 8, 6, 5, 4, 3, 2.5 is the payout

# ICM calculated by % of chip pool you have looking for first place chances.  
    # Then the ICM is calculated by assuming you do not win first; do the same with the remaining players.  
    # Assuming other combinations got first.
    
# ICM isn't calculated in small blind sizes until there are 12 opponents left
    # Therefore this program will ignore ICM completely.
    
    
################# Strategy #################
def get_randomized_strategies():
    strategies = []
    for i in range(6):
        strategy = get_randomized_strategy()
        strategies.append(strategy)
    return strategies

def get_randomized_strategy():
    strategy = []
    choices = [3, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20]
    length_of_strategy = 40
    for i in range(length_of_strategy):
        strategy.append(random.choices(choices)[0])
    return strategy




################# Monte Carlo #################
def get_average_profit_from_playing_hands(hands):
    profit = 0
    for i in range(hands):        
        profit_from_playing_hand = get_profit_from_playing_hand()
        profit += profit_from_playing_hand
    avg_profit = profit / hands
    return avg_profit



################# Playing a Hand #################
def get_profit_from_playing_hand(strategies):
    player_as_lj = get_randomized_player_as_lj()
    hands, deck = play_preflop(strategies, player_as_lj)
    hands, deck, my_winnings_given_postflop, board = play_flop(strategies, my_position_id, opponent_position_id, hands, deck)
    if my_winnings_given_postflop != None:
        return my_winnings_given_postflop    
    hands, deck, my_winnings_given_postflop, board = play_turn(strategies, my_position_id, opponent_position_id, hands, deck)
    if my_winnings_given_postflop != None:
        return my_winnings_given_postflop    
    my_winnings_given_postflop = play_river(strategies, my_position_id, opponent_position_id, hands, deck)
    return my_winnings_given_postflop

def get_randomized_player_as_lj():
    # 6 is BB, 1 is LJ
    return random.choices([1,2,3,4,5,6])[0]

def play_preflop(strategies, player_as_lj):
    player_as_lj_index = player_as_lj - 1
    players_remaining = 6
    raises = 0
    while players_remaining > 2 and raises <= 1: # Only want to continue with 1 raise, not 2.
        players_remaining = 6
        raises = 0        
        hands, deck = get_randomized_9_hands()
        raiser_position = None
        opponent_number = 0
        # LJ Turn
        action1 = get_players_first_preflop_action(1, raises, raiser_position, hands[0], strategies[(player_as_lj_index+0)%6])
        opponent_number += 1
        if action1 == "r":
            raises += 1
            raiser_position = 1
        elif action1 == "f":
            players_remaining -= 1
        # HJ Turn
        action2 = get_players_first_preflop_action(2, raises, raiser_position, hands[1], strategies[(player_as_lj_index+1)%6])
        opponent_number += 1
        if action2 == "r":
            raises += 1
            raiser_position = 2
        elif action2 == "f":
            players_remaining -= 1            
        # CO Turn
        action3 = get_players_first_preflop_action(3, raises, raiser_position, hands[2], strategies[(player_as_lj_index+2)%6])
        opponent_number += 1
        if action3 == "r":
            raises += 1
            raiser_position = 3
        elif action3 == "f":
            players_remaining -= 1            
        # BN Turn
        action4 = get_players_first_preflop_action(4, raises, raiser_position, hands[3], strategies[(player_as_lj_index+3)%6])
        opponent_number += 1
        if action4 == "r":
            raises += 1
            raiser_position = 4
        elif action4 == "f":
            players_remaining -= 1            
        # BN Turn
        action5 = get_players_first_preflop_action(5, raises, raiser_position, hands[4], strategies[(player_as_lj_index+4)%6])
        opponent_number += 1
        if action5 == "r":
            raises += 1
            raiser_position = 5
        elif action5 == "f":
            players_remaining -= 1            
        # BN Turn
        action6 = get_players_first_preflop_action(6, raises, raiser_position, hands[5], strategies[(player_as_lj_index+5)%6])
        opponent_number += 1
        if action6 == "r":
            raises += 1
            raiser_position = 6
        elif action6 == "f":
            players_remaining -= 1  
        if players_remaining > 2:
            print("Try again:", 'players_remaining:', players_remaining, [action1, action2, action3, action4, action5, action6])
    print("Result:", [action1, action2, action3, action4, action5, action6])
    print([print(hand) for hand in hands])
    return hands, deck
    
def get_players_first_preflop_action(seat_at_the_table, raises, raiser_position, hand, my_strategy):
    h = hand[2]
    suited = hand[3]
    paired = len(set(h)) == 1
    if raises == 0:
        if seat_at_the_table == 1:
            if suited:
                if h[1] >= 9:
                    return 'r'
                else:
                    return 'f'
            elif paired:
                if h[0] >= 5:
                    return 'r'
                else:
                    return 'f'
            else:
                if h in [[14,13], [14,12], [14,11], [13,12]]:
                    return 'r'
                else:
                    return 'f'
        elif seat_at_the_table == 2:
            if suited:
                if h[1] >= 9:
                    return 'r'
                else:
                    return 'f'  
            elif paired:
                return 'r'
            else:
                if h[1] >= 11:
                    return 'r'
                else:
                    return 'f'
        elif seat_at_the_table == 3:
            if suited:
                if (h[1] >= 9) or (h[0] == 14 and h[1] >= 5):
                    return 'r'
                else:
                    return 'f'  
            elif paired:
                return 'r'
            else:
                if h[1] >= 10:
                    return 'r'
                else:
                    return 'f'   
        elif seat_at_the_table == 4:
            if suited:
                if (h[1] >= 8) or (h[0] == 14) or (h[0]-h[1] <= 2 and h[1] != 2):
                    return 'r'
                else:
                    return 'f'  
            elif paired:
                return 'r'
            else:
                if h[1] >= 9:
                    return 'r'
                else:
                    return 'f' 
        elif seat_at_the_table == 5:
            if suited:
                if (h[1] >= 8) or (h[0] == 14) or (h[0]-h[1] <= 2 and h[1] != 2):
                    return 'r'
                else:
                    return 'f'  
            elif paired:
                return 'r'
            else:
                if (h[1] >= 9) or (h[0] == 14 and h[1] >= 5):
                    return 'r'
                else:
                    return 'f'
        elif seat_at_the_table == 6:
            if suited:
                if (h[1] >= 9):
                    return 'r'
                else:
                    return 'f'  
            elif paired:
                if h[0] >= 9:
                    return 'r'
                else:
                    return 'f'
            else:
                if (h[1] >= 12):
                    return 'r'
                else:
                    return 'f'
    elif raises == 1:
        if seat_at_the_table == 6 and raiser_position == 1:
            if suited:
                if h in [[14,13], [14,12]]:
                    return 'r'                
                elif h[1] >= 9:
                    return 'c'
                else:
                    return 'f'
            elif paired:
                if h[0] >= 12:
                    return 'r'                
                elif h[0] >= 2:
                    return 'c'
                else:
                    return 'f'
            else:
                if h == [14,13]:
                    return 'r'
                elif h[1] >= 11:
                    return 'c'                
                else:
                    return 'f'
        elif seat_at_the_table == 6 and raiser_position == 2:
            if suited:
                if h in [[14,13], [14,12], [14,11], [13,12]]:
                    return 'r'
                elif (h[1] >= 9) or (h[0] == 14 and h[1] >= 5):
                    return 'c'                
                else:
                    return 'f'
            elif paired:
                if h[0] >= 12:
                    return 'r'                
                elif h[0] >= 2:
                    return 'c'
                else:
                    return 'f'
            else:
                if h == [14,13]:
                    return 'r'
                elif h[1] >= 10:
                    return 'c'                
                else:
                    return 'f'
        elif seat_at_the_table == 6 and raiser_position == 3:
            if suited:
                if h in [[14,13], [14,12], [14,11], [13,12], [12,11], [11,10]]:
                    return 'r'
                elif (h[1] >= 8) or (h[0] == 14) or (h[0]-h[1] <= 2 and h[1] != 2):
                    return 'c'                
                else:
                    return 'f'
            elif paired:
                if h[0] >= 11:
                    return 'r'                
                elif h[0] >= 2:
                    return 'c'
                else:
                    return 'f'
            else:
                if h == [14,13]:
                    return 'r'
                elif h[1] >= 9:
                    return 'c'                
                else:
                    return 'f'
        elif seat_at_the_table == 6 and raiser_position == 4:
            if suited:
                if h[1] >= 10:
                    return 'r'
                elif (h[1] >= 8) or (h[0] == 14) or (h[0]-h[1] <= 2 and h[1] != 2):
                    return 'c'                
                else:
                    return 'f'
            elif paired:
                if h[0] >= 10:
                    return 'r'                
                elif h[0] >= 2:
                    return 'c'
                else:
                    return 'f'
            else:
                if h in [[14,13], [14,12], [13,12]]:
                    return 'r'
                elif (h[1] >= 9) or (h[0] == 14 and h[1] >= 5):
                    return 'c'                
                else:
                    return 'f'  
        elif seat_at_the_table == 6 and raiser_position == 5:
            if suited:
                if h[1] >= 9:
                    return 'r'
                elif (h[1] >= 4) or (h[0] == 14) or (h[0]-h[1] <= 2):
                    return 'c'                
                else:
                    return 'f'
            elif paired:
                if h[0] >= 9:
                    return 'r'                
                elif h[0] >= 2:
                    return 'c'
                else:
                    return 'f'
            else:
                if h in [[14,13], [14,12], [13,12]]:
                    return 'r'
                elif (h[1] >= 8):
                    return 'c'                
                else:
                    return 'f' 
        elif seat_at_the_table == 5 and raiser_position == 4:
            if suited:
                if h[1] >= 9:
                    return 'r'                            
                else:
                    return 'f'
            elif paired:
                if h[0] >= 5:
                    return 'r'
                else:
                    return 'f'
            else:
                if h in [[14,13], [14,12], [14,11], [13,12]]:
                    return 'r'            
                else:
                    return 'f'
        else:
            if suited:
                if (h == [14,13]) or (h in [[14,5], [14,4]]):
                    return 'r'
                elif h[1] >= 10:
                    return 'c'                
                else:
                    return 'f'
            elif paired:
                if h[0] >= 12:
                    return 'r'                
                elif h[0] >= 7:
                    return 'c'
                else:
                    return 'f'
            else:
                if h == [14,13]:
                    return 'r'
                elif (h == [14,12]):                    
                    return 'c'                
                else:
                    return 'f'            
    else:                
        if h in [[14,14], [13,13]]:
            return 'r'
        else:
            return 'f'
    
    
################# Shuffle and Deal #################
def get_randomized_9_hands():
    deck = shuffle_a_deck_of_cards()
    hands, deck = deal_two_cards_to_each(deck)
    return hands, deck

def shuffle_a_deck_of_cards():
    deck = []
    for i in range(2, 15):
        for j in range(1,5):
            deck.append([i, j])
    random.shuffle(deck)
    return deck
       
def deal_two_cards_to_each(deck):
    hands = [deck[0:2] + [sorted([deck[0][0], deck[1][0]], reverse=True)] + [deck[0][1] == deck[1][1]], 
             deck[2:4] + [sorted([deck[2][0], deck[3][0]], reverse=True)] + [deck[2][1] == deck[3][1]], 
             deck[4:6] + [sorted([deck[4][0], deck[5][0]], reverse=True)] + [deck[4][1] == deck[5][1]], 
             deck[6:8] + [sorted([deck[6][0], deck[7][0]], reverse=True)] + [deck[6][1] == deck[7][1]], 
             deck[8:10] + [sorted([deck[8][0], deck[9][0]], reverse=True)] + [deck[8][1] == deck[9][1]], 
             deck[10:12] + [sorted([deck[10][0], deck[11][0]], reverse=True)] + [deck[10][1] == deck[11][1]]]
    deck = deck[12:]
    return hands, deck

################# Flop #################
    
def play_flop(strategies, my_position_id, opponent_position_id, hands, deck):
    flop, deck = get_randomized_new_cards(deck=deck, number_of_cards=3)
    return hands, deck, my_winnings_given_postflop, board

def play_turn(strategies, my_position_id, opponent_position_id, hands, deck):
    turn, deck = get_randomized_new_cards(deck=deck, number_of_cards=1)
    return hands, deck, my_winnings_given_postflop, board

def play_river(strategies, my_position_id, opponent_position_id, hands, deck):
    river, deck = get_randomized_new_cards(deck=deck, number_of_cards=1)
    return my_winnings_given_postflop
    


def get_randomized_new_cards():
    pass




In [41]:
strategies = get_randomized_strategies()
# avg_profit = get_average_profit_from_playing_hands(hands=100)
hand, deck = play_preflop(strategies, 1)
hand, deck


Result: ['f', 'f', 'f', 'f', 'r', 'c']
[[7, 2], [10, 3], [10, 7], False]
[[14, 1], [9, 2], [14, 9], False]
[[9, 4], [10, 1], [10, 9], False]
[[4, 2], [8, 1], [8, 4], False]
[[9, 1], [12, 3], [12, 9], False]
[[6, 1], [6, 2], [6, 6], False]
[None, None, None, None, None, None]


([[[7, 2], [10, 3], [10, 7], False],
  [[14, 1], [9, 2], [14, 9], False],
  [[9, 4], [10, 1], [10, 9], False],
  [[4, 2], [8, 1], [8, 4], False],
  [[9, 1], [12, 3], [12, 9], False],
  [[6, 1], [6, 2], [6, 6], False]],
 [[14, 4],
  [3, 4],
  [14, 3],
  [4, 3],
  [12, 1],
  [8, 3],
  [10, 4],
  [11, 3],
  [8, 2],
  [4, 1],
  [11, 2],
  [11, 1],
  [7, 1],
  [6, 3],
  [11, 4],
  [6, 4],
  [2, 1],
  [5, 1],
  [12, 2],
  [13, 1],
  [13, 2],
  [7, 4],
  [3, 1],
  [5, 4],
  [3, 2],
  [4, 4],
  [7, 3],
  [5, 2],
  [9, 3],
  [5, 3],
  [8, 4],
  [3, 3],
  [2, 2],
  [2, 3],
  [10, 2],
  [12, 4],
  [2, 4],
  [14, 2],
  [13, 3],
  [13, 4]])

In [ ]:
deck = shuffle_a_deck_of_cards()
hands, deck = deal_two_cards_to_each(deck)
hands